# 🎬 YTautoma - YouTube Shorts Automation

Generate 60-second YouTube Shorts using local AI models.

| Component | Model |
|-----------|-------|
| Story | Gemma 3 (Ollama) |
| Images | Z-Image-Turbo |
| Video | Wan 2.2 |
| Voice | VibeVoice / Edge-TTS |

**Works on**: RunPod, Colab (A100), Lambda Labs

## 1️⃣ System Setup

In [1]:
# Install system dependencies
!apt-get update -qq && apt-get install -y -qq ffmpeg
!ffmpeg -version | head -1

W: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers


In [2]:
# Upgrade PyTorch (run this, then RESTART RUNTIME before next cell)
!pip install -q --upgrade typing_extensions torch torchvision
print('✅ Done! Now restart the runtime (Runtime > Restart runtime) and run the next cell')

✅ Done! Now restart the runtime (Runtime > Restart runtime) and run the next cell


In [1]:
# Verify PyTorch (run after restart)
import torch
print(f'PyTorch {torch.__version__} | CUDA: {torch.cuda.is_available()}')

PyTorch 2.9.1+cu128 | CUDA: True


In [2]:
!pip uninstall flash-attn -y && pip install "https://github.com/mjun0812/flash-attention-prebuild-wheels/releases/download/v0.6.8/flash_attn-2.7.4%2Bcu128torch2.9-cp312-cp312-linux_x86_64.whl"

Found existing installation: flash_attn 2.7.4+cu128torch2.9
Uninstalling flash_attn-2.7.4+cu128torch2.9:
  Successfully uninstalled flash_attn-2.7.4+cu128torch2.9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.1/406.1 MB 204.7 MB/s  0:00:0100:0100:01


## 2️⃣ Clone & Install

In [3]:
import os

# Auto-detect workspace
WORKSPACE = '/content' if os.path.exists('/content') else '/workspace' if os.path.exists('/workspace') else os.path.expanduser('~')
os.chdir(WORKSPACE)

# Clone repo
!git clone https://github.com/DragonLord1998/YTautoma.git 2>/dev/null || (cd YTautoma && git pull)
os.chdir('YTautoma')
PROJECT_DIR = os.getcwd()
print(f'Project: {PROJECT_DIR}')

remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 11 (delta 9), reused 7 (delta 5), pack-reused 0 (from 0)
Unpacking objects: 100% (11/11), 2.18 KiB | 26.00 KiB/s, done.
From https://github.com/DragonLord1998/YTautoma
   64ed38c..c8311f5  main       -> origin/main
Updating 64ed38c..c8311f5
Fast-forward
 Automator.ipynb                                    | 697 ++++-----------------
 .../services/wan_video_service.py                  |  15 +-
 2 files changed, 134 insertions(+), 578 deletions(-)
Project: /workspace/YTautoma


In [4]:
# Install Python dependencies
!pip install  -r requirements.txt
!pip install  git+https://github.com/huggingface/diffusers
!pip install  edge-tts easydict

  Using cached https://github.com/mjun0812/flash-attention-prebuild-wheels/releases/download/v0.6.8/flash_attn-2.7.4%2Bcu128torch2.9-cp312-cp312-linux_x86_64.whl (406.1 MB)
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-65h6_5fx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-65h6_5fx
  Resolved https://github.com/huggingface/diffusers to commit 17c0e79dbdf53fb6705e9c09cc1a854b84c39249
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.36.0.dev0-py3-none-any.whl size=4634817 sha256=171a729e42f54e4195b93f5caeb2dda910ab5e2d78dfc1337bfc5dd4f2bd74f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-4tg1_jqd/wheels/90/d4/44/a58bc00fb405fefb633b0d9d2307f6e3aec6cc1775d82555d3
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installatio

In [5]:
# Install Ollama + Gemma 3
!curl -fsSL https://ollama.com/install.sh | sh

import subprocess, time
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5)

!ollama pull gemma3:4b

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling aeda25e63ebd: 100% ▕██████████████████▏ 3.3 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling b6ae5839783f: 100% ▕██████████████████▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


In [6]:
# Setup Wan 2.2 (video generation)
!mkdir -p models
!git clone https://github.com/Wan-Video/Wan2.2.git models/Wan2.2 2>/dev/null || echo 'Already cloned'
!pip install  -r models/Wan2.2/requirements.txt
!pip install hf_transfer
# Download model (10GB) - uncomment to enable video generation
!huggingface-cli download Wan-AI/Wan2.2-TI2V-5B --local-dir models/Wan2.2-TI2V-5B

Already cloned
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached tokenizers-0.21.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
  Attempting uninstall: tokenizers━━━━━━━━━━━━━━ 0/3 [numpy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/3 [numpy]    WARNING: Ignoring invalid distribution ~orch (/usr/local/lib/python3.12/dist-packages)
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:━━━━━━━━━ 0

In [7]:
# Setup VibeVoice (high-quality TTS) - optional
!git clone https://github.com/microsoft/VibeVoice.git models/VibeVoice 2>/dev/null || echo 'Already cloned'
!cd models/VibeVoice && pip install  -e .

Already cloned
Obtaining file:///workspace/YTautoma/models/VibeVoice
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached accelerate-1.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached gradio-6.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiortc-1.14.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached fastapi-0.124.4-py3-none-any.whl.metadata (30 kB)
  Using cached cryptography-46.0.3-cp311-abi3-manylinux_2_34_x86_64.whl.metadata (5.7 kB)
  Using cached pyopenssl-25.3.0-py3-none-any.whl.metadata (17 kB)
  Using cached gradio_client-2.0.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached safehttpx-0.1.7-py3-none-any.whl.metadata (4.2 kB)
  Using cached typer-0.20.0-py3-none-any.whl.metadata (16 kB)
Using cached accelerate-1.6.0-py3-none-any.whl (354 kB)
Using cached aiortc-1.14.0-py3-none-any.whl (93 kB)
Usin

In [8]:
# Create .env configuration
import os
PROJECT_DIR = os.getcwd()

env_content = f"""OLLAMA_MODEL=gemma3:4b
OLLAMA_BASE_URL=http://localhost:11434

ZIMAGE_MODEL=Tongyi-MAI/Z-Image-Turbo
ZIMAGE_DEVICE=cuda

WAN_REPO_PATH={PROJECT_DIR}/models/Wan2.2
WAN_MODEL_PATH={PROJECT_DIR}/models/Wan2.2-TI2V-5B
WAN_T5_CPU=true
WAN_OFFLOAD_MODEL=true

TTS_ENGINE=edge
VIBEVOICE_REPO_PATH={PROJECT_DIR}/models/VibeVoice

LOW_VRAM_MODE=true
TORCH_DTYPE=float16
"""

with open('.env', 'w') as f:
    f.write(env_content)

print('✅ Configuration saved!')

✅ Configuration saved!


## 3️⃣ Generate YouTube Short

In [9]:
# Quick test: Story only
#!python main.py --story-only -c mystery

In [10]:
# Generate story + images
#!python main.py --images-only -c horror

In [11]:
#Do not forget to rename Wan2.2_VAE.pth to Wan2.1_VAE.pth

In [12]:
!pip install openai-whisper HyperPyYAML onnxruntime inflect wetext omegaconf conformer hydra-core lightning rich gdown matplotlib wget pyarrow pyworld librosa decord modelscope GitPython decord peft onnxruntime pandas matplotlib -e  git+https://github.com/facebookresearch/sam2.git@0e78a118995e66bb27d78518c4bd9a3e95b4e266#egg=SAM-2  loguru sentencepiece

Obtaining SAM-2 from git+https://github.com/facebookresearch/sam2.git@0e78a118995e66bb27d78518c4bd9a3e95b4e266#egg=SAM-2
  Skipping because already up-to-date.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for SAM-2 (pyproject.toml) ... done
  Created wheel for SAM-2: filename=sam_2-1.0-0.editable-cp312-cp312-linux_x86_64.whl size=11501 sha256=06a5a28bdf64718c5ad322b86706f634c87ff013aa289c64f35f1368c7559008
  Stored in directory: /tmp/pip-ephem-wheel-cache-zxl8c599/wheels/10/a7/62/87dd08be5abdd14305ee69bff5571cb332fe0f6f12014a24a9
Successfully built SAM-2
  Attempting uninstall: SAM-2
    Found existing installation: SAM-2 1.0
    Uninstalling SAM-2-1.0:
      Successfully uninstalled SAM-2-1.0


In [13]:
!pip install torch torchvision torchaudio opencv-python diffusers transformers tokenizers accelerate tqdm imageio[ffmpeg] easydict ftfy dashscope 

In [14]:
!pip install --upgrade transformers peft diffusers

  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached diffusers-0.36.0-py3-none-any.whl.metadata (20 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
Using cached diffusers-0.36.0-py3-none-any.whl (4.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.36.0.dev0
    Uninstalling diffusers-0.36.0.dev0:
      Successfully uninstalled diffusers-0.36.0.dev0
  Attempting uninstall: transformers━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [diffusers]
   ━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [diffusers]    WARNING: Ignoring invalid distribution ~orch (/us

In [15]:
# Full pipeline (requires Wan 2.2 model download)
!pip install ftfy
!pip install decord
!pip install librosa
!pip install opencv-python-headless

!python main.py -c sci-fi

  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4

🎬 YOUTUBE SHORTS AUTOMATION PIPELINE

📖 STEP 1: Story Generation
----------------------------------------
📖 Topic: [sci-fi] Time stops for everyone except one person
🤖 Calling Ollama (gemma3:4b)...
✅ Generated: 'Echoes of Stillness' with 6 scenes
💾 Saved to: /workspace/YTautoma/output/project_20251214_153314/story.json
   ✅ Story: 'Echoes of Stillness'
   📄 File: /workspace/YTautoma/output/project_20251214_153314/story.json

🎨 STEP 2: Visual Generation
----------------------------------------
✅ Wan 2.2 ready at /workspace/YTautoma/models/Wan2.2

🎬 Processing 6 scenes for 'Echoes of Stillness'

🔄 Loading Z-Image model...
🔄 Loading Z-Image model

## 4️⃣ View & Download

In [ ]:
# View generated images
from IPython.display import Image, display
import glob

images = sorted(glob.glob('output/**/base_image.png', recursive=True))[-6:]
for img in images:
    print(img.split('/')[-2])
    display(Image(filename=img, width=250))

In [ ]:
# Download outputs
import os, glob

try:
    from google.colab import files
    !cd output && zip -r ../output.zip .
    files.download('output.zip')
except ImportError:
    print('Find outputs at: ./output/')
    !find output -name '*.mp4' -o -name '*.png' | head -20